# BoW  : Bag of Word

## keras Tokenizer 활용

여기서 단어장(Vocabulary) 이라는 또 다른 개념을 이해해 보겠습니다. 문헌에 따라 사전 또는 단어 집합이라고 불리는데요. 단어장이란 중복을 제거한 단어들의 집합을 말합니다. 이는 Bag of Words랑은 다른 개념입니다. 위의 예제에서는 단어장의 크기가 10개네요!


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print(">> Bag of Words :", bow) # bow 출력
print('>> 단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수
print("-"*20)
print(">> Word_index :",tokenizer.word_index,)
print(">> Word_index len :",len(tokenizer.word_index))

>> Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
>> 단어장(Vocabulary)의 크기 : 10
--------------------
>> Word_index : {'likes': 1, 'to': 2, 'watch': 3, 'movies': 4, 'mary': 5, 'john': 6, 'too': 7, 'also': 8, 'football': 9, 'games': 10}
>> Word_index len : 10


## CountVectorizer

- input : a collection of text documents (list of sentences, paragraphs, or even entire books)
- function : tokenize, n-grams or counts the frequency  
- output : row(documents) x columns(words,tokens) shaped sparse matrix

## scikit-learn CountVectorizer 활용

CounterVectorizer의 vector.fit_transform(입력 문장).toarray()으로 출력되는 Bag of Words 결과를 보면 각 단어의 빈도만 출력될 뿐, 어떤 단어의 빈도인지는 나오지 않습니다. [[1 1 1 1 3 2 2 2 1 2]]이라는 값만으로는 각 숫자가 어떤 단어의 빈도인지 알 수가 없지요?

이는 vector.vocabulary_를 통해서 각 단어에 부여된 인덱스를 확인하면 되는데요. 주의할 점은 이 인덱스는 0부터 시작한다는 점입니다. 가령, also의 인덱스는 0이네요. 그런데 BoW 결과 리스트에서 0번 인덱스에 해당하는 값은 1이니까, also의 빈도는 1입니다. 또는 likes의 인덱스는 4인데, BoW에서 4번 인덱스에 해당하는 값은 3이니까 likes의 빈도는 3임을 알 수 있지요!

그리고 이 Bag of Words의 단어장의 크기는 아래와 같이 구하면 되겠지요!

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))


Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}
단어장(Vocabulary)의 크기 : 10


단어장이라는 개념은 Bag of Words를 위해 사용되는 개념이라기보다는, 자연어 처리에서 보편적으로 쓰이는 용어입니다. 그런데 이를 계속 강조하는 이유는 지금부터 사용할 DTM(Document-Term Matrix) 이라는 개념 이해를 돕기 위해서입니다. 지금 Bag of Words를 배우면서 이런 생각이 들지 않으셨나요? 그래서 이 개념을 도대체 어디서 사용할 수 있다는 거지? 이를 이해하기 위해서 DTM으로 가보겠습니다!

# DTM과 코사인 유사도

DTM(Document-Term Matrix) 은 직역하면 문서-단어 행렬입니다. DTM은 여러 문서의 Bag of Words를 하나의 행렬로 구현한 것인데요. 다시 말해 DTM은 각 문서에 등장한 단어의 빈도수를 하나의 행렬로 통합시킵니다. DTM은 문서를 행으로, 단어를 열로 가지는 행렬이지만, 문헌에 따라서는 열을 문서로 하고 단어를 행으로 하여 TDM(Term-Document Matrix) 이라고 부르기도 합니다.

예를 들어, 아래와 같은 3개의 문서가 있다고 해보겠습니다.

- Doc 1: Intelligent applications creates intelligent business processes
- Doc 2: Bots are intelligent applications
- Doc 3: I do business intelligence

이 3개의 문서로부터 얻을 수 있는 DTM은 아래와 같습니다.

content img
[출처 : https://www.darrinbishop.com/blog/2017/10/text-analytics-document-term-matrix/]
위의 DTM을 보면 각 행은 각 문서를 나타내고 있습니다. 그리고 각 열은 문서 3개의 통합 단어장(Vocabuary)에 있는 단어들로 구성되어 있습니다. 이렇게 구성하면 앞서 문서 1개로 BoW를 만들었을 때와는 달리, 각 행에는 0이 많이 포함됩니다.

이때 각 행을 문서 벡터(document vector), 열을 단어 벡터(word vector)라고 부를 수 있는데요. 문서의 수가 많아지면 많아질수록, 통합 단어장의 크기도 커지게 되어서 DTM은 결국 문서 벡터와 단어 벡터 모두 대부분의 값이 0이 되는 성질을 가지고 있습니다.

여러 문서들로 DTM을 만들었다면, 이제 Bag of Words로 무엇을 할 수 있는지를 이해할 차례입니다. DTM을 사용하면 각 문서들을 비교하고, 각 문서 간 유사도를 구할 수 있게 됩니다. 예를 들어볼까요?

문서1 : I like dog
문서2 : I like cat
문서3 : I like cat I like cat

위와 같이 3개의 문서가 있다고 해볼게요. 각 단어에 대한 인덱스를 cat은 0, dog는 1, I는 2, like는 3이라고 했을 때 생성되는 DTM의 형태는 아래와 같습니다.


이제 각 문서 벡터의 유사도를 구해보겠습니다. 저자는 가장 보편적으로 사용되는 유사도 계산 방법인 코사인 유사도를 택했습니다. 아래와 같이 각 문서 벡터와 코사인 유사도 함수를 정의합니다.

In [3]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

DTM에서 코사인 유사도는 0 이상 1 이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단할 수 있습니다.
문서1, 문서2, 문서3의 각 문서 벡터에 대해서 코사인 유사도를 계산해 보겠습니다.

In [4]:
print('{:.2f}'.format(cos_sim(doc1, doc2))) #문서1과 문서2의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc1, doc3))) #문서1과 문서3의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc2, doc3))) #문서2과 문서3의 코사인 유사도

0.67
0.67
1.00


문서1과 문서2의 코사인 유사도는 0.67, 문서1과 문서3의 코사인 유사도 또한 0.67입니다. 그런데 문서2와 문서3의 코사인 유사도는 1입니다. 이는 한 문서 내의 모든 단어의 빈도수가 똑같이 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 1이 되기 때문입니다.

## scikit-learn CountVectorizer 활용

사이킷런의 CountVectorizer를 이용하여 DTM을 만드는 방법은 앞서 Bag of Words를 만드는 방법과 동일합니다. 단지, 다수의 문서를 입력값으로 주면 됩니다. 다음은 3개의 문서에 대해서 DTM을 만드는 과정을 보여줍니다.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


# DTM or TDM

DTM의 한계점
이쯤에서 DTM의 성질을 정리해보도록 하겠습니다.

DTM은 bag of words 기반으로 문서를 비교할 수 있는 행렬입니다.

그런데 DTM은 두 가지 한계점을 지닙니다.

1. 먼저, DTM에서 문서의 수와 단어의 수가 계속 늘어날수록, 행과 열은 대부분의 값이 0을 가진다는 특징이 있습니다. 이는 저장 공간 측면에서 낭비입니다. (워드 임베딩 챕터에서 배우겠지만, 지나친 차원의 크기는 차원의 저주라는 또 다른 문제를 발생시킵니다.)

2. 두 번째 한계점은 단어의 빈도에만 집중하는 방법 자체의 한계입니다. 예를 들어, 영어 데이터를 가지고 DTM을 만들었을 때, 불용어 'the'는 어떤 문서에서도 자주 등장하는 경향이 있습니다. 그런데 문서1과 문서2에서 둘 다 'the'가 많이 등장했다고 하여 이 두 문서가 유사한 문서라고 봐도 될까요?

사실 여러 문서에는 중요한 단어와 중요하지 않은 단어들이 혼재되어 있는데, 'the'와 같이 아무 문서에나 자주 등장하는 단어들은 문서끼리 비교할 때 그다지 도움이 되지 않습니다. 그렇다면 중요한 단어와 중요하지 않은 단어에 가중치를 따로 선별해서 주는 방법은 없을까요?


# TF-IDF

TF = DTM이니 단어 빈도에 대한 DTM을 구해두고

거기에 IDF만 다시  구해서 곱해주면 됨. (IDF = Weight)

예제) 문서가 5개고, 2개에서만 like 단어가 등장, 2번문서에서 200번, 3번문서에서 300번 등장했다. 
2,3번 문서에서의 like의 TF-IDF는 얼마인가? 



- DF (문서의 빈도) : 2 (2개문서에서 등장했으므로)﻿
- N (전체 문서의 수) : 5
- IDF : log 5/2 (약 0.92)
- 2번문서 like의 TF-IDF : 200 * 0.92 = 약 183
- 3번문서 like의 TF-IDF : 300 * 0.92 = 약 274

TF-IDF의 목적이 특정 문서에서만 자주 등장하는 단어는 중요도는 높다고 판단한다는 것이었으므로,

1번문서에서 한번도 등장하지 않았던 like가 3번문서에 300번이나 등장하니 중요하다고 할 수 있음.

In [6]:
from math import log
import pandas as pd

docs = [
  'John likes to watch movies and Mary likes movies too',  # 문서 3개라고 가정
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

우선 DTM의 열을 만들기 위해서 문서 3개의 단어들이 모두 들어간 통합 단어장을 만들겠습니다.

In [7]:
vocab = list(set(w for doc in docs for w in doc.split())) # set하면 중복이 없어짐
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


현재 단어장의 크기는 13이네요. 중복을 제거한 단어들의 개수가 13개라는 의미입니다. 이제 총 문서의 수를 변수 N에다가 저장하겠습니다. 우리는 이미 이 값이 3이라는 사실을 알고 있지요?

In [8]:
N = len(docs) # 총 문서의 수
N

3

이제 TF 함수, IDF 함수, TF-IDF 함수를 만들겠습니다.

여기서 IDF를 실제 구현할 때는 앞서 배운 식과는 다소 다른 식을 사용하게 됩니다. 실제로 많은 파이썬 패키지들이 앞에서 배운 식에서 조금씩 조정된, 서로 다른 식을 사용합니다.

우선, 여기서는 log 항의 분모에 1을 더해주겠습니다. 이는 (Bag of Words를 사용할 때 일반적인 상황은 아니지만) 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지하기 위함입니다. 

그리고 log 항에 1을 더해주겠습니다. 이는 log 항의 분자와 분모값이 동일해졌을 때, 
log의 진수가 1이 됨에 따라서 IDF의 값이 0이 되는 것을 방지하기 위함입니다.

In [9]:
def tf(t, d):
    return d.count(t)
 
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc    
    return log(N/(df + 1)) + 1
 
def tfidf(t, d):
    return tf(t,d)* idf(t)

In [10]:
# tf 함수로 DTM 만들기

result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tf(t, d))
        
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [11]:
# 각 단어의 IDF 구하기

result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


IDF는 'likes'나 'to' 그리고 'watch'와 같이 모든 문서에 등장한 단어가 가장 낮은 값을 가집니다. 그리고 그다음으로 두 개의 문서에 등장한 'Mary'가 그다음으로 낮은 값을 가지며, 그 외의 1개의 문서에만 등장한 단어들은 가장 높은 값을 가지는 것을 볼 수 있습니다. 이제 TF-IDF 행렬을 출력해봅시다. DTM에 있는 각 단어의 TF에 각 단어의 IDF를 곱해준 값입니다.

In [12]:
# TF-IDF 행렬 출력하기 (TF * IDF)

result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


scikit-learn TFidVectorizer 활용
사이킷런으로 DTM을 만들기 위해서는 CountVectorizer를 사용했었습니다. 이와 유사하게 사이킷런에서는 TF-IDF를 자동으로 계산하여 출력하는 TfidfVectorizer를 제공합니다.

향후 실습 중에 결괏값이 예상과 달라 혼란이 일어나는 일이 없도록 미리 언급하자면, 사이킷런의 TfidfVectorizer 또한 이번에 배운 기본식에서 조정된 식을 사용합니다. 간단히 언급하면, TfidfVectorizer는 위에서 우리가 파이썬으로 구현한 식에서 더 나아가 
log 항의 분자에도 1을 더해주며, TF-IDF의 결과에 L2 Norm까지 추가로 수행한다는 점이 다릅니다.

사이킷런의 TfidfVectorizer를 통해 TF-IDF 행렬을 출력해봅시다.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


# LSA (Latent Semantic Analysis)

한국어로 잠재 의미 분석이라고 불리는 LSA(Latent Semantic Analysis) 는 전체 코퍼스에서 문서 속 단어들 사이의 관계를 찾아내는 자연어 처리 정보 검색 기술입니다.

LSA를 사용하면 
- 단어와 단어 사이, 
- 문서와 문서 사이, 
- 단어와 문서 사이의 
의미적 유사성 점수를 찾아낼 수 있습니다. 그리고 이 방법은 어떤 문서에서 특정 단어들의 빈도가 몇인지를 판단하는 것보다 효과적인 경우가 많습니다.

## 특잇값 분해 (SVD)

LSA를 이해하려면 먼저 선형대수학의 특잇값 분해(Singular Value Decompotion) 에 대해 이해할 필요가 있습니다.
우선 행렬에 대한 용어를 파악해야 합니다. 아래 문서에 있는 부록 : 행렬의 종류에서 단위행렬, 역행렬, 직교 행렬, 정방 행렬의 내용을 읽어보시길 바랍니다.

In [14]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
import os

csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('/aiffel/aiffel/topic_modelling/data/abcnews-date-text.csv',
 <http.client.HTTPMessage at 0x7f8c0e5f0640>)

In [17]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

약 108만 개의 샘플이 존재합니다. 5개의 샘플만 출력해봅시다.

In [18]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


publish_date는 이번 실습에 불필요하므로 headline_text만 별도로 저장합니다.

In [19]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [20]:
# 중복확인
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

In [21]:
# 중복제거
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

## 데이터 정제 및 정규화
이제 텍스트 데이터를 정제 및 정규화하는 과정을 진행해 보겠습니다. 우선 NLTK의 토크나이저를 이용해 전체 텍스트 데이터에 대해서 단어 토큰화를 수행하고, NLTK가 제공하는 불용어 리스트를 사용하여 불용어를 제거합니다.

아래 과정은 다소(몇 분 정도) 시간이 걸릴 수 있으니 당황하지 마세요.

In [22]:
print(len(text['headline_text']))

1054983


In [23]:
# NLTK 토크나이저를 이용해서 토큰화해서 컬럼에 다시 넣고
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

In [24]:
# 불용어 제거해준 컬럼
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [25]:
print(len(text['headline_text']))

1054983


이제 동일한 단어지만 다른 표현을 가지는 단어들을 하나의 단어로 통합(lemmatization)하는 단어 정규화 과정, 그리고 길이가 1 ~ 2인 단어를 제거하는 전처리를 진행합니다.
- 단어 정규화 : 동사원형으로 바꾸기

In [26]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


In [27]:
print(len(text))

1054983


## 역토큰화 및 DTM 생성
DTM을 생성하는 CountVectorizer 또는 TF-IDF 행렬을 생성하는 TfidfVectorizer의 입력으로 사용하기 위해서 토큰화 과정을 역으로 되돌리는 역토큰화(detokenization) 를 수행해보겠습니다.

In [28]:
# 토큰화 된 것을 blank로 join 하여 문장으로 만듬
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc

전처리 최종 결과는 train_data에 저장하였습니다. 5개의 샘플을 출력해봅시다.

In [29]:
print(type(train_data))
train_data[:5]

<class 'list'>


['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [30]:
print(text.shape)
print(type(text))

(1054983,)
<class 'pandas.core.series.Series'>


전처리 최종 결과인 train_data는 다음에 배울 LDA 실습에서도 재사용 할 예정입니다. CountVectorizer를 사용하여 DTM을 생성해봅시다. 단어의 수는 5,000개로 제한하겠습니다.

In [31]:
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

DTM을 생성했습니다. DTM의 크기를 확인해봅시다.

In [32]:
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


DTM의 크기(shape)는 (문서의 수 × 단어 집합의 크기)입니다.

## scikit-learn TruncatedSVD 활용
이제 Truncated SVD를 통해 LSA를 수행해 봅시다. 토픽의 수를 10으로 정하겠습니다. 이는 앞서 배운 하이퍼파라미터 k에 해당되며, 행렬 Vk가
k × (단어의 수)의 크기를 가지도록 DTM에 TruncatedSVD를 수행합니다.

In [33]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20290521e-02, -3.66748861e-03,  1.82185847e-02, ...,
         4.10651367e-04, -1.93677392e-03, -6.16085838e-03],
       [ 2.90324014e-02, -1.08050960e-02,  1.81323692e-02, ...,
         4.19283558e-04, -5.53088309e-03,  8.60715777e-03],
       [ 5.03183260e-03, -1.99935131e-03,  9.76862282e-03, ...,
        -3.05135668e-03,  1.40747617e-03, -1.52025054e-03],
       ...,
       [ 2.96972834e-02,  4.32264152e-03,  2.50307698e-02, ...,
         1.97805961e-02,  1.00294957e-02,  1.88620676e-02],
       [ 6.13701684e-02, -5.47536391e-03,  1.35733468e-01, ...,
         7.51587659e-01,  9.21644677e-01, -2.33290375e-01],
       [ 7.14392625e-02,  2.88425372e-02,  5.46664453e-04, ...,
        -5.04999166e-03, -2.64799783e-02, -5.05442888e-03]])

TruncatedSVD를 통해 얻은 행렬 Vk의 크기를 확인해봅시다.

In [34]:
print(lsa_model.components_.shape)

(10, 5000)


## 토픽 모델링(Topic Modelling) using LSA
전체 코퍼스로부터 주요 주제인 토픽을 찾아내는 실습을 해봤습니다. 이처럼 문서의 집합에서 토픽을 찾아내는 프로세스를 토픽 모델링(Topic Modelling) 이라고 합니다. 이는 고객의 소리와 같이 많은 문서에서 주요 주제를 알아내는 일이 중요할 때 사용합니다.  
행렬 Vk가 k × (단어의 수) 의 크기를 가지는 것을 확인할 수 있습니다. 이제 각 행 (원래는 행이 문서, 문장, 열이 단어)을 전체 코퍼스의 k개의 주제(topic)로 판단하고 각 주제에서 n개씩 단어를 출력해 봅시다.

In [35]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74636), ('man', 0.45354), ('charge', 0.21095), ('new', 0.1409), ('court', 0.11143)]
Topic 2: [('man', 0.69416), ('charge', 0.30045), ('court', 0.16798), ('face', 0.11337), ('murder', 0.1063)]
Topic 3: [('new', 0.83664), ('plan', 0.23658), ('say', 0.18265), ('govt', 0.11052), ('council', 0.10987)]
Topic 4: [('say', 0.7391), ('plan', 0.35779), ('govt', 0.16657), ('council', 0.12706), ('urge', 0.07863)]
Topic 5: [('plan', 0.73253), ('council', 0.17672), ('govt', 0.14245), ('urge', 0.08444), ('water', 0.06465)]
Topic 6: [('govt', 0.50404), ('court', 0.27863), ('urge', 0.22684), ('fund', 0.19417), ('face', 0.16141)]
Topic 7: [('charge', 0.58017), ('court', 0.37851), ('face', 0.35462), ('plan', 0.13387), ('murder', 0.09821)]
Topic 8: [('win', 0.58167), ('court', 0.40735), ('kill', 0.20756), ('crash', 0.19527), ('face', 0.15188)]
Topic 9: [('win', 0.62709), ('charge', 0.45398), ('australia', 0.10241), ('cup', 0.0846), ('world', 0.07754)]
Topic 10: [('kill', 0.4985), ('c

# 토픽 모델링(Topic Modeling)

토픽 모델링은 문서 집합에서 주제를 찾아내기 위한 기술
토픽 모델링은 '특정 주제에 관한 문서에서는 특정 단어가 자주 등장할 것이다'라는 직관을 기반
 예를 들어, 주제가 '개'인 문서에서는 개의 품종, 개의 특성을 나타내는 단어가 다른 문서에 비해 많이 등장
 주로 사용되는 토픽 모델링 방법은 잠재 의미 분석과 잠재 디리클레 할당 기법이 있음

# 5. LDA (Latent Dirichlet Allocation : 잠재 디리클레 할당) 
LSA와 함께  토픽 모델링의 또 다른 대표적인 알고리즘.

모든 단어는 하나의 토픽에 속하고, 임의의 단어가 있을 때 이것이 어떤 토픽이 될 껀지 찾아내는 알고리즘

'나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야.'

주제와 문서 / 주제와 단어간의 확률로 토픽 추출.

LDA 실습은 LSA 실습 과정에서 만들어둔 전처리가 완료된 train_data 전처리, 정규화 다한것)를 재사용한다고 가정합니다. 해당 데이터는 역토큰화 과정을 거친 데이터입니다.

## TF-IDF 행렬 생성
LDA는 DTM 또는 TF-IDF를 입력으로 받을 수 있습니다. 여기서는 TF-IDF를 사용하겠습니다.

TfidfVectorizer를 사용하여 TF-IDF 행렬을 생성해봅시다. 단어의 수는 5,000개로 제한하겠습니다. TF-IDF 행렬을 생성한 후에는 행렬의 크기를 확인합시다.

In [36]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


## scikit-learn LDA Model 활용
사이킷런의 LDA 모델을 사용하여 학습합니다. LSA와 마찬가지로 동일한 사이킷런 패키지이므로 앞으로 진행되는 실습 과정은 LSA와 매우 유사합니다. 토픽의 개수는 10개로 정했습니다. 이는 n_components의 인자값입니다. 시간이 수분 정도 소요될 것이므로 느긋하게 기다려 주세요.

In [37]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.17024867, 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.0366096 , 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26687206, 0.02914502, 0.02914502, ..., 0.13007484, 0.02916018,
        0.28739608],
       [0.10378115, 0.02637829, 0.12325014, ..., 0.02637829, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.2255442 , ..., 0.03376055, 0.03376055,
        0.03376055]])

LDA를 통해 얻은 결과 행렬의 크기를 확인해봅시다.

In [38]:
print(lda_model.components_.shape)

(10, 5000)


전체 코퍼스로부터 얻은 10개의 토픽과 각 토픽에서의 단어의 비중을 보겠습니다.

LSA 실습의 코드를 참고해, 아래 코드 블록에 직접 코드를 작성해 주세요.

In [39]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합시다
terms = tfidf_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)

Topic 1: [('australia', 9359.06334), ('sydney', 5854.97288), ('attack', 4784.76322), ('change', 4193.63035), ('year', 3924.88997)]
Topic 2: [('government', 6344.07413), ('charge', 5947.12292), ('man', 4519.7974), ('state', 3658.16422), ('live', 3625.10473)]
Topic 3: [('australian', 7666.65651), ('say', 7561.01807), ('police', 5513.22932), ('home', 4048.38409), ('report', 3796.04446)]
Topic 4: [('melbourne', 5298.35047), ('south', 4844.59835), ('death', 4281.78433), ('china', 3214.44581), ('women', 3029.28443)]
Topic 5: [('win', 5704.0914), ('canberra', 4322.0963), ('die', 4025.63057), ('open', 3771.65243), ('warn', 3577.47151)]
Topic 6: [('court', 5246.3124), ('world', 4536.86331), ('country', 4166.34794), ('woman', 3983.97748), ('crash', 3793.50267)]
Topic 7: [('election', 5418.5038), ('adelaide', 4864.95604), ('house', 4478.6135), ('school', 3966.82676), ('2016', 3955.11155)]
Topic 8: [('trump', 8189.58575), ('new', 6625.2724), ('north', 3705.40987), ('rural', 3521.42659), ('donald',